In [24]:
import os
import time
import zipfile
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    make_scorer, precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.random_projection import SparseRandomProjection

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            from sklearn.random_projection import johnson_lindenstrauss_min_dim
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Decision Tree with Manual Cross-Validation

def flastThreshold(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps, combination_label, params):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define Decision Tree model with given parameters
    dt_model = DecisionTreeClassifier(
        criterion=params.get('criterion', 'entropy'),
        max_depth=params.get('max_depth', None),
        min_samples_split=params.get('min_samples_split', 2),
        min_samples_leaf=params.get('min_samples_leaf', 1),
        max_features=params.get('max_features', None),
        random_state=42
    )

        # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize metrics storage for each threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_threshold = {threshold: {'f1': 0, 'accuracy': 0, 'precision': 0, 'recall': 0, 'mcc': 0}
                             for threshold in thresholds}
    successFold = 0

    for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
        X_train, X_test = Z[train_index], Z[test_index]
        y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

        if sum(y_train) == 0 or sum(y_test) == 0:
            print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
            continue

        # Train the model
        dt_model.fit(X_train, y_train)

        # Predict probabilities on test set
        y_pred_proba = dt_model.predict_proba(X_test)

        # Calculate metrics for each threshold
        for threshold in thresholds:
            y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

            # Calculate metrics for this threshold
            f1 = f1_score(y_test, y_pred, zero_division=1)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, zero_division=1)
            recall = recall_score(y_test, y_pred, zero_division=1)
            mcc = matthews_corrcoef(y_test, y_pred)

            # Accumulate metrics for the current threshold
            metrics_per_threshold[threshold]['f1'] += f1
            metrics_per_threshold[threshold]['accuracy'] += accuracy
            metrics_per_threshold[threshold]['precision'] += precision
            metrics_per_threshold[threshold]['recall'] += recall
            metrics_per_threshold[threshold]['mcc'] += mcc

        successFold += 1

    if successFold == 0:
        print("No valid folds. Exiting.")
        return params, None

    # Average metrics over all folds
    for threshold in thresholds:
        metrics_per_threshold[threshold]['f1'] /= successFold
        metrics_per_threshold[threshold]['accuracy'] /= successFold
        metrics_per_threshold[threshold]['precision'] /= successFold
        metrics_per_threshold[threshold]['recall'] /= successFold
        metrics_per_threshold[threshold]['mcc'] /= successFold

    # Save the results for each threshold
    outFile = f"{combination_label}-params-dt-{n_splits}-folds-Threshold.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        # Write the header
        fo.write("threshold,accuracy,precision,recall,f1,mcc\n")
        # Write the data for each threshold
        for threshold in thresholds:
            fo.write(f"{threshold},{metrics_per_threshold[threshold]['accuracy']},{metrics_per_threshold[threshold]['precision']},"
                     f"{metrics_per_threshold[threshold]['recall']},{metrics_per_threshold[threshold]['f1']},{metrics_per_threshold[threshold]['mcc']}\n")

    print(f"Decision Tree analysis completed for {successFold} folds. Results saved to: {outFile}")
    return params, metrics_per_threshold


  


if __name__ == "__main__":

    params = {
        "criterion": "entropy",
        "max_depth": 300,
        "min_samples_split": 5,
        "min_samples_leaf": 1,
        "max_features" : 'log2'
    }

    # Parameters setup for the first combination
    flakyZip = "compressedDataset/flaky_files.zip"
    nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"
    largerNonFlakyZip = "compressedDataset/all_nonflaky_files.zip"

    # Create separate result directories for equal and larger non-flaky combinations
    outDirEqual = "results/equal_flaky_nonflaky/"
    outDirLarger = "results/larger_nonflaky/"
    os.makedirs(outDirEqual, exist_ok=True)
    os.makedirs(outDirLarger, exist_ok=True)

    # Create separate extract directories for each combination to avoid file confusion
    extractDirEqual = "extracted/equal_flaky_nonflaky/"
    extractDirLarger = "extracted/larger_nonflaky/"
    os.makedirs(extractDirEqual, exist_ok=True)
    os.makedirs(extractDirLarger, exist_ok=True)

    # Perform Decision Tree analysis for the first combination (flaky vs smaller non-flaky)
    print("Starting Decision Tree analysis for flaky vs smaller non-flaky files (47 each)...")
    best_params_5folds_1, best_score_5folds_1 = flastThreshold(
        outDirEqual, flakyZip, nonFlakyZip, extractDirEqual, 5, dim=100, eps=0.3, combination_label="equal", params=params)
    best_params_3folds_1, best_score_3folds_1 = flastThreshold(
        outDirEqual, flakyZip, nonFlakyZip, extractDirEqual, 3, dim=100, eps=0.3, combination_label="equal", params=params)

    print("Best results for 5-fold on equal combination:")
    print(f"Best Parameters: {best_params_5folds_1}")
    print(f"Best F1 Score: {best_score_5folds_1}")

    print("Best results for 3-fold on equal combination:")
    print(f"Best Parameters: {best_params_3folds_1}")
    print(f"Best F1 Score: {best_score_3folds_1}")

    # Perform Decision Tree analysis for the second combination (flaky vs larger non-flaky)
    print("Starting Decision Tree analysis for flaky vs larger non-flaky files...")
    best_params_5folds_2, best_score_5folds_2 = flastThreshold(
        outDirLarger, flakyZip, largerNonFlakyZip, extractDirLarger, 5, dim=100, eps=0.3, combination_label="larger", params=params)
    best_params_3folds_2, best_score_3folds_2 = flastThreshold(
        outDirLarger, flakyZip, largerNonFlakyZip, extractDirLarger, 3, dim=100, eps=0.3, combination_label="larger", params=params)

    print("Best results for 5-fold on larger non-flaky combination:")
    print(f"Best Parameters: {best_params_5folds_2}")
    print(f"Best F1 Score: {best_score_5folds_2}")

    print("Best results for 3-fold on larger non-flaky combination:")
    print(f"Best Parameters: {best_params_3folds_2}")
    print(f"Best F1 Score: {best_score_3folds_2}")




Starting Decision Tree analysis for flaky vs smaller non-flaky files (47 each)...
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Decision Tree analysis completed for 5 folds. Results saved to: equal-params-dt-5-folds-Threshold.csv
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Decision Tree analysis completed for 3 folds. Results saved to: equal-params-dt-3-folds-Threshold.csv
Best results for 5-fold on equal combination:
Best Parameters: {'criterion': 'entropy', 'max_depth': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2'}
Best F1 Score: {0.1: {'f1': 0.6185994397759104, 'accuracy': 0.6485380116959065, 'precision': 0.7146464646464646, 'recall': 0.5800000000000001, 'mcc': 0.3256764910632027}, 0.2: {'f1': 0.6185994397759104, 'accuracy': 0.6485380116959065, 'precision': 0.7146464646464646, 'recall': 0.5800000000000001, 'mcc': 0.3256764910632027}, 0.30000000000000004: {'

In [16]:
import os
import time
import zipfile
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    make_scorer, precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
)
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.random_projection import SparseRandomProjection

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            from sklearn.random_projection import johnson_lindenstrauss_min_dim
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Decision Tree with Manual Cross-Validation

def flastThreshold(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps, combination_label, params):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define Decision Tree model with given parameters
    dt_model = DecisionTreeClassifier(
        criterion=params.get('criterion', 'entropy'),
        max_depth=params.get('max_depth', None),
        min_samples_split=params.get('min_samples_split', 2),
        min_samples_leaf=params.get('min_samples_leaf', 1),
        max_features=params.get('max_features', None),
        random_state=42
    )

       # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize storage for metrics per fold and threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_fold = {fold+1: {threshold: {'f1': 0, 'accuracy': 0, 'precision': 0, 'recall': 0, 'mcc': 0}
                                 for threshold in thresholds}
                        for fold in range(n_splits)}
    successFold = 0

    for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
        X_train, X_test = Z[train_index], Z[test_index]
        y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

        if sum(y_train) == 0 or sum(y_test) == 0:
            print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
            continue

        # Train the model
        dt_model.fit(X_train, y_train)

        # Predict probabilities on test set
        y_pred_proba = dt_model.predict_proba(X_test)

        # Calculate metrics for each threshold for this fold
        for threshold in thresholds:
            y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

            # Calculate metrics for this threshold
            f1 = f1_score(y_test, y_pred, zero_division=1)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, zero_division=1)
            recall = recall_score(y_test, y_pred, zero_division=1)
            mcc = matthews_corrcoef(y_test, y_pred)

            # Store metrics for the current threshold for this fold
            metrics_per_fold[fold+1][threshold]['f1'] = f1
            metrics_per_fold[fold+1][threshold]['accuracy'] = accuracy
            metrics_per_fold[fold+1][threshold]['precision'] = precision
            metrics_per_fold[fold+1][threshold]['recall'] = recall
            metrics_per_fold[fold+1][threshold]['mcc'] = mcc

        successFold += 1

    if successFold == 0:
        print("No valid folds. Exiting.")
        return params, None

    # Save the results for each threshold and fold
    outFile = f"{combination_label}-params-dt-{n_splits}-folds-Threshold-allKfold.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        # Write the header
        fo.write("fold,threshold,accuracy,precision,recall,f1,mcc\n")
        # Write the data for each fold and each threshold
        for fold in range(1, successFold + 1):
            for threshold in thresholds:
                fo.write(f"{fold},{threshold},{metrics_per_fold[fold][threshold]['accuracy']},{metrics_per_fold[fold][threshold]['precision']},"
                         f"{metrics_per_fold[fold][threshold]['recall']},{metrics_per_fold[fold][threshold]['f1']},{metrics_per_fold[fold][threshold]['mcc']}\n")

    print(f"Decision Tree analysis completed for {successFold} folds. Results saved to: {outFile}")
    return params, metrics_per_fold



if __name__ == "__main__":

    params = {
        "criterion": "entropy",
        "max_depth": 300,
        "min_samples_split": 5,
        "min_samples_leaf": 1,
        "max_features" : 'log2'
    }

    # Parameters setup for the first combination
    flakyZip = "compressedDataset/flaky_files.zip"
    nonFlakyZip = "compressedDataset/reduced_nonflaky_files.zip"
    largerNonFlakyZip = "compressedDataset/all_nonflaky_files.zip"

    # Create separate result directories for equal and larger non-flaky combinations
    outDirEqual = "results/equal_flaky_nonflaky/"
    outDirLarger = "results/larger_nonflaky/"
    os.makedirs(outDirEqual, exist_ok=True)
    os.makedirs(outDirLarger, exist_ok=True)

    # Create separate extract directories for each combination to avoid file confusion
    extractDirEqual = "extracted/equal_flaky_nonflaky/"
    extractDirLarger = "extracted/larger_nonflaky/"
    os.makedirs(extractDirEqual, exist_ok=True)
    os.makedirs(extractDirLarger, exist_ok=True)

    # Perform Decision Tree analysis for the first combination (flaky vs smaller non-flaky)
    print("Starting Decision Tree analysis for flaky vs smaller non-flaky files (47 each)...")
    best_params_5folds_1, best_score_5folds_1 = flastThreshold(
        outDirEqual, flakyZip, nonFlakyZip, extractDirEqual, 5, dim=100, eps=0.3, combination_label="equal", params=params)
    best_params_3folds_1, best_score_3folds_1 = flastThreshold(
        outDirEqual, flakyZip, nonFlakyZip, extractDirEqual, 3, dim=100, eps=0.3, combination_label="equal", params=params)

    print("Best results for 5-fold on equal combination:")
    print(f"Best Parameters: {best_params_5folds_1}")
    print(f"Best F1 Score: {best_score_5folds_1}")

    print("Best results for 3-fold on equal combination:")
    print(f"Best Parameters: {best_params_3folds_1}")
    print(f"Best F1 Score: {best_score_3folds_1}")

    # Perform Decision Tree analysis for the second combination (flaky vs larger non-flaky)
    print("Starting Decision Tree analysis for flaky vs larger non-flaky files...")
    best_params_5folds_2, best_score_5folds_2 = flastThreshold(
        outDirLarger, flakyZip, largerNonFlakyZip, extractDirLarger, 5, dim=100, eps=0.3, combination_label="larger", params=params)
    best_params_3folds_2, best_score_3folds_2 = flastThreshold(
        outDirLarger, flakyZip, largerNonFlakyZip, extractDirLarger, 3, dim=100, eps=0.3, combination_label="larger", params=params)

    print("Best results for 5-fold on larger non-flaky combination:")
    print(f"Best Parameters: {best_params_5folds_2}")
    print(f"Best F1 Score: {best_score_5folds_2}")

    print("Best results for 3-fold on larger non-flaky combination:")
    print(f"Best Parameters: {best_params_3folds_2}")
    print(f"Best F1 Score: {best_score_3folds_2}")



Starting Decision Tree analysis for flaky vs smaller non-flaky files (47 each)...
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Decision Tree analysis completed for 5 folds. Results saved to: equal-params-dt-5-folds-Threshold-allKfold.csv
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Decision Tree analysis completed for 3 folds. Results saved to: equal-params-dt-3-folds-Threshold-allKfold.csv
Best results for 5-fold on equal combination:
Best Parameters: {'criterion': 'entropy', 'max_depth': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2'}
Best F1 Score: {1: {0.1: {'f1': 0.6, 'accuracy': 0.5789473684210527, 'precision': 0.6, 'recall': 0.6, 'mcc': 0.15555555555555556}, 0.2: {'f1': 0.6, 'accuracy': 0.5789473684210527, 'precision': 0.6, 'recall': 0.6, 'mcc': 0.15555555555555556}, 0.30000000000000004: {'f1': 0.6, 'accuracy': 0.5789473684210527, 'precision': 0.6, 'recal

Number of flaky documents: 47
Number of non-flaky documents: 254
Total number of documents: 301
Decision Tree analysis completed for 5 folds. Results saved to: larger-params-dt-5-folds-Threshold-allKfold.csv
Number of flaky documents: 47
Number of non-flaky documents: 254
Total number of documents: 301
Decision Tree analysis completed for 3 folds. Results saved to: larger-params-dt-3-folds-Threshold-allKfold.csv
Best results for 5-fold on larger non-flaky combination:
Best Parameters: {'criterion': 'entropy', 'max_depth': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2'}
Best F1 Score: {1: {0.1: {'f1': 0.35294117647058826, 'accuracy': 0.819672131147541, 'precision': 0.42857142857142855, 'recall': 0.3, 'mcc': 0.2573637971418306}, 0.2: {'f1': 0.35294117647058826, 'accuracy': 0.819672131147541, 'precision': 0.42857142857142855, 'recall': 0.3, 'mcc': 0.2573637971418306}, 0.30000000000000004: {'f1': 0.35294117647058826, 'accuracy': 0.819672131147541, 'precision': 0